# Similitud entre productos

Nesse notebook, iremos comparar a similaridade entre títulos de produtos do mercado livre, analisando se a solução proposta é escalável e o tempo necessário para execução da função.

## Carregamento dos dados

In [59]:
import pandas as pd

titles_test = pd.read_csv("items_titles_test.csv", encoding="utf-8")
titles = pd.read_csv("items_titles.csv", encoding="utf-8")

print(f"Titles test contains {len(titles_test)} items.")
print(f"Titles contains {len(titles)} items.")

Titles test contains 10000 items.
Titles contains 30000 items.


In [60]:
titles_test.head(5)

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


In [61]:
titles.head(5)

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


## Pretratamento dos dados

No pretratamento dos dados, caracteres especiais, símbolos e números são removidos e os textos são colocados em minúsculo. Depois disso são removidas palavras com pouco conteúdo semântico (stopwords), e as palavras são substituídas pelas suas raízes, fazendo com que o resultado não seja afetado por conjugação de verbos e plurais de substantivos.

In [62]:
import re

def preprocess_title(title: str) -> str:
    # Remove unidentified char and lower text
    filtered_string = title.replace("", "").lower()
    # Remove non char (e.g. digits and punctuation)
    processed_title = re.sub(r"[^a-zà-öø-ÿ\s]", " ", filtered_string)
    processed_title = re.sub(r"\s{2,}", " ", processed_title)
    return processed_title.strip()

In [63]:
titles_test.loc[:5, "ITE_ITEM_TITLE"].apply(preprocess_title)

0       tênis olympikus esporte valente masculino kids
1    bicicleta barra forte samy c marchas cubo c ro...
2              tênis usthemp slip on temático labrador
3          tênis casual feminino moleca tecido tie dye
4            tênis star baby sapatinho conforto brinde
5                  tênis oakley frequency preto marrom
Name: ITE_ITEM_TITLE, dtype: object

In [64]:
import nltk

# Download stemmer and stopwords from nltk
nltk.download('rslp')
nltk.download('stopwords')

stemmer = nltk.stem.RSLPStemmer()
stopwords = nltk.corpus.stopwords.words('portuguese')

def text_to_stems(text: str) -> list[str]:
    tokens = text.split(" ")
    stems = [stemmer.stem(token) for token in tokens if token not in stopwords] 
    return stems

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\jeh_m\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeh_m\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
titles_test.loc[:5,"ITE_ITEM_TITLE"].apply(preprocess_title).apply(text_to_stems)

0     [tênil, olympiku, esport, valent, masculin, kid]
1    [biciclet, barr, fort, samy, c, march, cub, c,...
2               [tênil, usthemp, slip, on, temá, labr]
3          [tênil, cas, feminin, molec, tec, tie, dye]
4             [tênil, st, baby, sapat, confort, brind]
5             [tênil, oakley, frequency, pret, marrom]
Name: ITE_ITEM_TITLE, dtype: object

## Solução 1

### Similaridade de texto

A primeira solução faz a contagem de palavras comuns entre duas setenças e considera a similaridade a razão entre o número de palavras comuns e o número de palavras na maior sentença.

In [66]:
def calculate_title_similarity(title_1: str, title_2: str) -> float:
    text_1, text_2 = preprocess_title(title_1), preprocess_title(title_2)
    stems_1, stems_2 = text_to_stems(text_1), text_to_stems(text_2)
    set_1, set_2 = set(stems_1), set(stems_2)
    return len(set_1 & set_2) / max(len(set_1), len(set_2))

In [67]:
calculate_title_similarity("Zapatillas Nike", "Zapatillas Adidas")

0.5

In [68]:
calculate_title_similarity("Zapatillas Nike", "Zapatillas Nike")

1.0

### Análise de desempenho

Essa solução tem complexidade O(n^2), já que é necessário comparar os items aos pares. Realizando a comparação para o dataset de teste, temos:

In [69]:
titles_test_values =  titles_test["ITE_ITEM_TITLE"].values
res_1_test: list[dict[str, str|float]] = []

for t1 in titles_test_values[:1000]:
    for t2 in titles_test_values[:1000]:
        if t2 < t1:
            continue

        res_1_test.append({
            "ITE_ITEM_TITLE_1": t1,
            "ITE_ITEM_TITLE_2": t2,
            "SCORE": calculate_title_similarity(t1, t2),
        })

pd.DataFrame(res_1_test).head(5)

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,SCORE
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino Kids,1.000000
1,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Usthemp Slip-on Temático - Labrador 2,0.166667
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Star Baby Sapatinho Conforto + Brinde,0.166667
3,Tênis Olympikus Esporte Valente - Masculino Kids,Under Armour Hovr Phantom 2 Conexão Bluetooth ...,0.125000
4,Tênis Olympikus Esporte Valente - Masculino Kids,Versão Coreana Da Tendência Dos Sapatos Casuai...,0.000000


Para apenas 1000 amostras do dataset de teste, o algoritmo teve tempo de execução de aproximadamente 3,5 min. Para os datasets com 10k e 30k, dado a complexidade O(n^2), isso resultaria 350 minutos e 3150 minutos, sendo assim inviável. 

## Solução 2

A segunda solução faz uso do TFIDF, uma técnica usual para computar a importância de palavras em documentos a partir da frequência que elas aparecem. Com ela, cálculo de similaridade é feito de forma matricial, sendo mais eficiente que a solução 1.

In [74]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

titles_test_values = [" ".join(text_to_stems(preprocess_title(title))) for title in titles_test["ITE_ITEM_TITLE"].values]

vectorizer = TfidfVectorizer()
test_tfidf_matrix = vectorizer.fit_transform(titles_test_values)
cosine_sim_test = cosine_similarity(test_tfidf_matrix, test_tfidf_matrix)

mask = np.triu(np.ones(cosine_sim_test.shape, dtype=bool))
cosine_sim_test[~mask] = np.nan

res_2_test = pd.DataFrame(cosine_sim_test, index=titles_test["ITE_ITEM_TITLE"].values, columns=titles_test["ITE_ITEM_TITLE"].values)
res_2_test = res_2_test.stack().dropna().reset_index().rename(columns={'level_0': 'ITE_ITEM_TITLE_1', 'level_1': 'ITE_ITEM_TITLE_2', 0: 'SCORE'})
res_2_test.dropna(how="any")

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,SCORE
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino Kids,1.000000
1,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,0.000000
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Usthemp Slip-on Temático - Labrador 2,0.014983
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Casual Feminino Moleca Tecido Tie Dye,0.014527
4,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Star Baby Sapatinho Conforto + Brinde,0.016531
...,...,...,...
50004995,Tênis Feminino Infantil Molekinha Tie Dye,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,0.041615
50004996,Tênis Feminino Infantil Molekinha Tie Dye,Tênis Polo Ralph Lauren Modelo Cantor Low Bran...,0.012183
50004997,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,1.000000
50004998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,Tênis Polo Ralph Lauren Modelo Cantor Low Bran...,0.008301


In [75]:
res_2_test.to_csv("ejercicio_2_results.csv", index=False)

Os resultados para o dataset de teste foram calculados em um minuto e não foi possível calcular os resultados para o dataset completo por falta de espaço na memória. Entretanto, como esses resultados são calculados através de operação matricial, é possível utilizar arquitetura distribuída, portanto a solução é escalável.

## Referências

[The Optimization of Fuzzy String Matching Using TF-IDF and Nearest Neighbors Algorithm (Medium)](https://audhiaprilliant.medium.com/fuzzy-string-matching-optimization-using-tf-idf-and-knn-b07fce69b58f#:~:text=The%20comparison%20to%20Levenshtein%20distance&text=Compared%20to%20the%20fuzzy%20string,with%20the%20number%20of%20data.)